In [ ]:
# Adjusted process to use only available 5-day forecast data
from datetime import datetime, timedelta
from collections import defaultdict
import requests
import pandas as pd

# OpenWeatherMap API key
API_KEY = "09806c8ab9fb92886b5a722c6359df79"

# District coordinates for weather fetching
districts = {
    "동구": (36.3241, 127.4349),
    "중구": (36.317, 127.418),
    "서구": (36.3528, 127.3823),
    "유성구": (36.3832, 127.3308),
    "대덕구": (36.3617, 127.4199),
}

# Weather condition mapping
weather_mapping = {
    "clear sky": "맑음",
    "few clouds": "흐림",
    "scattered clouds": "흐림",
    "broken clouds": "흐림",
    "overcast clouds": "흐림",
    "rain": "비",
    "light rain": "비",
    "moderate rain": "비",
    "heavy rain": "비",
    "shower rain": "비",
    "snow": "눈",
    "light snow": "눈",
    "heavy snow": "눈",
    "fog": "안개",
    "mist": "안개",
    "haze": "안개",
}

weather_types = ["맑음", "흐림", "비", "눈", "안개"]
default_weather = "기타"

# Map weather descriptions to Korean terms
def map_weather_description(description):
    return weather_mapping.get(description, default_weather)

# Fetch weather data for a specific latitude and longitude
def get_weather(lat, lon):
    url = f"http://api.openweathermap.org/data/2.5/forecast"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": API_KEY,
        "units": "metric",  # Use Celsius
        "lang": "en",
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching weather data: {response.status_code}, {response.text}")
        return None

# Analyze weather data for available dates
def analyze_weather(data):
    weather_counts = defaultdict(lambda: defaultdict(int))
    for item in data['list']:
        date = item['dt_txt'].split()[0]
        description = item['weather'][0]['description']
        mapped_weather = map_weather_description(description)
        weather_counts[date][mapped_weather] += 1
    return weather_counts

# Main process
results = {}
for district, (lat, lon) in districts.items():
    weather_data = get_weather(lat, lon)
    if weather_data:
        results[district] = analyze_weather(weather_data)

# Aggregate results for each district
final_results = {district: defaultdict(int, {weather: 0 for weather in weather_types}) for district in districts.keys()}
for district, weather_data in results.items():
    for date, counts in weather_data.items():
        for weather_type, count in counts.items():
            final_results[district][weather_type] += count

# Display results
result_df = pd.DataFrame(final_results).fillna(0).astype(int)
result_df.index.name = "날씨 유형"
result_df.columns.name = "지역"
result_df = result_df.T



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the CSV file
file_path = 'your_file_path.csv'  # Replace with your file path
data = pd.read_csv(file_path)
population_data = {
    "대덕구": 337416,
    "동구": 437280,
    "서구": 917806,
    "유성구": 738622,
    "중구": 449064
}

# Preprocessing
# Extract month from '년월' and convert to numeric
data['월'] = data['년월'].str[-2:].astype(int)

# Select input features and target variable
features = ['맑음', '흐림', '비', '눈', '안개', '인구수', '월']
target = 'ARI'

# Drop rows with missing values
cleaned_data = data.dropna(subset=features + [target])

# Separate features and target
X = cleaned_data[features]
y = cleaned_data[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16, verbose=1)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Mean Squared Error: {loss}, Mean Absolute Error: {mae}")

# Predict ARI values for test data
y_pred = model.predict(X_test)


In [ ]:
# Add population data to result_df
result_df["인구수"] = result_df.index.map(population_data)

# Add a "월" column (assuming the prediction is for the next month, e.g., 2 for February)
result_df["월"] = next_month  # Replace with the appropriate month if needed

# Prepare data for prediction
X_predict = result_df.reset_index(drop=True)

# Ensure the columns match the trained model's input
features = ["맑음", "흐림", "비", "눈", "안개", "인구수", "월"]
X_predict = X_predict[features]

# Use the trained model to predict ARI values
y_pred = model.predict(X_predict)

# Add predictions to the result DataFrame
result_df["Predicted ARI"] = y_pred

# Display the final DataFrame with predictions
result_df

In [ ]:
result_df["Predicted ARI"][0]


In [ ]:
import pandas as pd

# Load the cleaned data
import pandas as pd

# Try different encodings to read the file
file_path = 'Final_Cleaned_Dataset.csv'

try:
    final_cleaned_data = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        final_cleaned_data = pd.read_csv(file_path, encoding='euc-kr')  # Common for Korean text
    except UnicodeDecodeError:
        final_cleaned_data = pd.read_csv(file_path, encoding='latin1')  # Fallback encoding


result_df = pd.DataFrame({
    "지역": ["동구", "중구", "서구", "유성구", "대덕구"],
    "Predicted ARI": [
        result_df["Predicted ARI"][0],  # Use the first prediction dynamically
        result_df["Predicted ARI"][1],  # Placeholder or dynamic value
        result_df["Predicted ARI"][2],  # Placeholder or dynamic value
        result_df["Predicted ARI"][3],  # Placeholder or dynamic value
        result_df["Predicted ARI"][4]   # Placeholder or dynamic value
    ]
})

# Ensure column names and data types match
final_cleaned_data['지역'] = final_cleaned_data['지역'].astype(str).str.strip()


# Merge the datasets
final_cleaned_data = final_cleaned_data.merge(
    result_df[["지역", "Predicted ARI"]],
    left_on="지역",
    right_on="지역",
    how="left"
).drop(columns=["지역"])

# Display the updated dataset
print(final_cleaned_data.head())


In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement


# 데이터 불러오기 함수 정의
def load_data(file_path):
    try:
        df = pd.read_csv(file_path, encoding='utf-8-sig')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(file_path, encoding='euc-kr')
        except UnicodeDecodeError as e:
            print("파일을 읽는 데 실패했습니다.")
            print(e)
            exit()
    return df

# 데이터 필터링 및 필요한 열 선택 함수 정의
def filter_data(df):
    required_columns = ['경도', '위도', 'Predicted ARI']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"누락된 열이 있습니다: {missing_columns}")
        exit()
    
    result_df = df[required_columns].dropna(subset=['경도', '위도', 'Predicted ARI'])
    result_df.rename(columns={'Predicted ARI': 'Predicted ARI'}, inplace=True)
    print(f"경도, 위도, ARI 값이 있는 데이터 수: {len(result_df)}")
    return result_df

# ARI 값에 따라 색상 및 단계 계산 함수 정의
def get_color_and_stage(ari, min_ari, max_ari):
    normalized_value = (ari - min_ari) / (max_ari - min_ari)
    colors = ["#00FF00", "#FFFF00", "#FFA500", "#FF0000"]
    stages = [1, 2, 3, 4]
    
    for i, threshold in enumerate([0.25, 0.5, 0.75, 1.0]):
        if normalized_value <= threshold:
            return colors[i], stages[i]
    
    return colors[-1], stages[-1]  # 최대값 초과 시 마지막 단계 반환

# 지도 생성 및 표시 함수 정의
def create_map(df):
    center_lat = df['위도'].mean()
    center_lon = df['경도'].mean()
    mymap = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    
    max_ari = df['Predicted ARI'].max()
    min_ari = df['Predicted ARI'].min()
    
    for idx, row in df.iterrows():
        color, stage = get_color_and_stage(row['Predicted ARI'], min_ari, max_ari)
        
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=f"Predicted ARI: {row['Predicted ARI']:.2f}, 단계: {stage}"
        ).add_to(mymap)
    
    # 범례 추가
    legend_html = '''
    {% macro html(this, kwargs) %}
    <div style="position: fixed; bottom: 50px; right: 50px; width: 120px; height: 120px; 
                border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
                ">&nbsp; <b>ARI 단계</b> <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FF0000"></i> 4단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FFA500"></i> 3단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#FFFF00"></i> 2단계 <br>
      &nbsp; <i class="fa fa-circle fa-1x" style="color:#00FF00"></i> 1단계 <br>
    </div>
    {% endmacro %}
    '''
    
    macro = MacroElement()
    macro._template = Template(legend_html)
    mymap.get_root().add_child(macro)
    
    return mymap

# 메인 함수
def main():
    # Update file_path with the correct dataset path
    result_df = final_cleaned_data
    
    if result_df.empty:
        print("필터링된 데이터가 없습니다. 프로그램을 종료합니다.")
        return
    
    mymap = create_map(result_df)
    return mymap

# 실행
mymap = main()
mymap
